# Pre-Entrega – Etapa 1: Recopilación y Preparación de Datos
Datasets:
- `clientes.csv`
- `marketing.csv`
- `ventas.csv`



In [25]:
#Librerías e importaciones base
import numpy as np
import io
from urllib.request import urlopen

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

In [26]:
#Definición de URLs
urls = {
    "clientes":  "https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/clientes.csv",
    "marketing": "https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/marketing.csv",
    "ventas":    "https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/ventas.csv",
}
urls

{'clientes': 'https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/clientes.csv',
 'marketing': 'https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/marketing.csv',
 'ventas': 'https://raw.githubusercontent.com/elkingdavis/Analisis_de_datos_PY_202510_EDV/main/ventas.csv'}

In [27]:
def read_csv_from_url(url: str) -> pd.DataFrame:
    return pd.read_csv(url)

clientes  = read_csv_from_url(urls["clientes"])
marketing = read_csv_from_url(urls["marketing"])
ventas    = read_csv_from_url(urls["ventas"])

clientes.head(), marketing.head(), ventas.head()

(   id_cliente               nombre  edad         ciudad  ingresos
 0           1      Aloysia Screase    44  Mar del Plata  42294.68
 1           2  Kristina Scaplehorn    25        Posadas  24735.04
 2           3       Filip Castagne    50    Resistencia  35744.85
 3           4          Liuka Luard    39   Bahía Blanca  27647.96
 4           5        Dore Cockshtt    28        Rosario  28245.65,
    id_campanha         producto  canal  costo fecha_inicio   fecha_fin
 0           74  Adorno de pared     TV   4.81   20/03/2024  03/05/2024
 1           12           Tablet   RRSS   3.40   26/03/2024  13/05/2024
 2           32  Lámpara de mesa  Email   5.54   28/03/2024  20/04/2024
 3           21       Smartphone   RRSS   6.37   29/03/2024  16/05/2024
 4           58         Alfombra  Email   4.25   31/03/2024  05/05/2024,
    id_venta           producto   precio  cantidad fecha_venta          categoria
 0       792  Cuadro decorativo   $69.94       5.0  02/01/2024         Decoración


In [28]:
def quick_eda(df: pd.DataFrame, name: str):
    print(f"\n {name.upper()} ")
    print("Shape:", df.shape)
    print("\nInfo:")
    print(df.info())
    print("\nNulos por columna:\n", df.isna().sum())
    if df.select_dtypes(include=np.number).shape[1] > 0:
        print("\nDescriptivos (numéricos):\n", df.describe().T)

quick_eda(clientes,  "clientes")
quick_eda(marketing, "marketing")
quick_eda(ventas,    "ventas")


 CLIENTES 
Shape: (567, 5)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_cliente  567 non-null    int64  
 1   nombre      567 non-null    object 
 2   edad        567 non-null    int64  
 3   ciudad      567 non-null    object 
 4   ingresos    567 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 22.3+ KB
None

Nulos por columna:
 id_cliente    0
nombre        0
edad          0
ciudad        0
ingresos      0
dtype: int64

Descriptivos (numéricos):
             count          mean           std     min       25%       50%      75%       max
id_cliente  567.0    284.000000    163.823075    1.00    142.50    284.00    425.5    567.00
edad        567.0     37.940035     10.202885   20.00     30.00     37.00     43.0     81.00
ingresos    567.0  34668.739012  12974.531446  170.29  26015.24  35066.83  42457.1  8

In [29]:
import re

def _find_first(df: pd.DataFrame, patterns):
    cols = list(df.columns)
    for p in patterns:
        for c in cols:
            if re.search(p, c, flags=re.IGNORECASE):
                return c
    return None

def detect_date_col(df: pd.DataFrame):
    patterns = [r"fecha", r"date", r"fechaventa", r"created", r"fec_", r"dt_"]
    return _find_first(df, patterns)

def detect_amount_col(df: pd.DataFrame):
    patterns = [r"monto", r"importe", r"total", r"amount", r"precio", r"price", r"valor"]
    return _find_first(df, patterns)

def detect_id(df: pd.DataFrame, prefer):
    return _find_first(df, prefer)

def try_parse_dates(df: pd.DataFrame, col: str):
    if col and col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
    return df

In [30]:
COL_FECHA_VENTAS  = None  # ejemplo: 'fecha'
COL_MONTO_VENTAS  = None  # ejemplo: 'monto' o 'total'
COL_ID_CLIENTE_V  = None  # ejemplo: 'cliente_id'
COL_ID_CLIENTE_C  = None  # ejemplo: 'cliente_id'
COL_ID_CAMPANA_V  = None  # ejemplo: 'campana_id' o 'campaign_id'
COL_ID_CAMPANA_M  = None  # ejemplo: 'campana_id' o 'campaign_id'

if COL_FECHA_VENTAS is None:
    COL_FECHA_VENTAS = detect_date_col(ventas)
if COL_MONTO_VENTAS is None:
    COL_MONTO_VENTAS = detect_amount_col(ventas)
if COL_ID_CLIENTE_V is None:
    COL_ID_CLIENTE_V = detect_id(ventas,  [r"cliente", r"customer", r"id_cli", r"idcliente", r"client_id"])
if COL_ID_CLIENTE_C is None:
    COL_ID_CLIENTE_C = detect_id(clientes, [r"cliente", r"customer", r"id_cli", r"idcliente", r"client_id"])
if COL_ID_CAMPANA_V is None:
    COL_ID_CAMPANA_V = detect_id(ventas,  [r"camp", r"promo", r"mkt", r"campaign"])
if COL_ID_CAMPANA_M is None:
    COL_ID_CAMPANA_M = detect_id(marketing,[r"camp", r"promo", r"mkt", r"campaign"])

print("Columnas inferidas/forzadas:")
print({
    'fecha_ventas': COL_FECHA_VENTAS,
    'monto_ventas': COL_MONTO_VENTAS,
    'id_cliente_ventas': COL_ID_CLIENTE_V,
    'id_cliente_clientes': COL_ID_CLIENTE_C,
    'id_campana_ventas': COL_ID_CAMPANA_V,
    'id_campana_marketing': COL_ID_CAMPANA_M,
})

Columnas inferidas/forzadas:
{'fecha_ventas': 'fecha_venta', 'monto_ventas': 'precio', 'id_cliente_ventas': None, 'id_cliente_clientes': 'id_cliente', 'id_campana_ventas': None, 'id_campana_marketing': 'id_campanha'}


In [31]:
ventas = try_parse_dates(ventas, COL_FECHA_VENTAS)
if COL_MONTO_VENTAS and COL_MONTO_VENTAS in ventas.columns:
    ventas[COL_MONTO_VENTAS] = pd.to_numeric(ventas[COL_MONTO_VENTAS], errors='coerce')

df = ventas.copy()
if COL_ID_CLIENTE_V and COL_ID_CLIENTE_C and COL_ID_CLIENTE_V in ventas.columns and COL_ID_CLIENTE_C in clientes.columns:
    df = df.merge(clientes, left_on=COL_ID_CLIENTE_V, right_on=COL_ID_CLIENTE_C, how='left', suffixes=('', '_cli'))

if COL_ID_CAMPANA_V and COL_ID_CAMPANA_M and COL_ID_CAMPANA_V in df.columns and COL_ID_CAMPANA_M in marketing.columns:
    df = df.merge(marketing, left_on=COL_ID_CAMPANA_V, right_on=COL_ID_CAMPANA_M, how='left', suffixes=('', '_mkt'))

print("Shape final tras integraciones:", df.shape)
df.head()

Shape final tras integraciones: (3035, 6)


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,NaN,5.0,2024-01-02,Decoración
1,811,Lámpara de mesa,NaN,5.0,2024-01-02,Decoración
2,1156,Secadora,NaN,3.0,2024-01-02,Electrodomésticos
3,1372,Heladera,NaN,8.0,2024-01-02,Electrodomésticos
4,1546,Secadora,NaN,4.0,2024-01-02,Electrodomésticos


In [32]:
if not COL_FECHA_VENTAS or not COL_MONTO_VENTAS:
    raise ValueError("No se detectaron columnas de fecha y/o monto. Seteá COL_FECHA_VENTAS y COL_MONTO_VENTAS arriba.")

col_fecha = COL_FECHA_VENTAS
col_monto = COL_MONTO_VENTAS

df['_anio_mes'] = df[col_fecha].dt.to_period('M').astype(str)
ventas_mensuales = (
    df.groupby('_anio_mes')
      .agg(total_ventas=(col_monto, 'sum'),
           operaciones=(col_monto, 'count'),
           ticket_promedio=(col_monto, 'mean'))
      .reset_index()
      .sort_values('_anio_mes')
)
ventas_mensuales.head(12)

,_anio_mes,total_ventas,operaciones,ticket_promedio
0,2024-01,0.0,0,NaN
1,2024-02,0.0,0,NaN
2,2024-03,0.0,0,NaN
3,2024-04,0.0,0,NaN
4,2024-05,0.0,0,NaN
5,2024-06,0.0,0,NaN
6,2024-07,0.0,0,NaN
7,2024-08,0.0,0,NaN
8,2024-09,0.0,0,NaN
9,2024-10,0.0,0,NaN


In [33]:
# Lista: top 5 meses por ventas
top5_meses = ventas_mensuales.sort_values('total_ventas', ascending=False)['_anio_mes'].head(5).tolist()

# Tupla: métrica resumen global (total, promedio, operaciones)
resumen_global = (
    float(df[col_monto].sum()),
    float(df[col_monto].mean()),
    int(df[col_monto].count())
)

clientes_unicos = set()
if COL_ID_CLIENTE_V and COL_ID_CLIENTE_V in df.columns:
    clientes_unicos = set(df[COL_ID_CLIENTE_V].dropna().astype(str))

ticket_por_mes = dict(zip(ventas_mensuales['_anio_mes'], ventas_mensuales['ticket_promedio']))

print("Top 5 meses por ventas (lista):", top5_meses)
print("Resumen global (tupla: total, promedio, n):", resumen_global)
print("Cantidad de clientes únicos (set):", len(clientes_unicos))
print("Ejemplo de diccionario mes->ticket_promedio (primeros 5):", dict(list(ticket_por_mes.items())[:5]))

Top 5 meses por ventas (lista): ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05']
Resumen global (tupla: total, promedio, n): (0.0, nan, 0)
Cantidad de clientes únicos (set): 0
Ejemplo de diccionario mes->ticket_promedio (primeros 5): {'2024-01': nan, '2024-02': nan, '2024-03': nan, '2024-04': nan, '2024-05': nan}


In [34]:
problemas = {}
if df[col_monto].lt(0).any():
    problemas['montos_negativos'] = int(df[col_monto].lt(0).sum())
k_cols = [c for c in [COL_FECHA_VENTAS, COL_MONTO_VENTAS, COL_ID_CLIENTE_V] if c]
nulos = df[k_cols].isna().mean().to_dict() if k_cols else {}
print({"problemas": problemas, "porc_nulos": nulos})

{'problemas': {}, 'porc_nulos': {'fecha_venta': 0.0, 'precio': 1.0}}
